# Battery project - Day 2

Today we will calculate the energy cost/gain associated with intercalating a lithium atom into graphite using approaches at different levels of theory. After today you should be able to:

- Setup structures and do relaxations using ASE and GPAW.
- Discuss which level of theory is required to predict the Li intercalation energy and why?

The Li intercalation reaction is:
$$Li(s) + C_6^{graphite} \rightarrow LiC_6$$
and the intercalation energy will then be
$$E_{Li@graphite} = E_{LiC_6} - (E_{Li(s)} + E_{C_6^{graphite}})$$
We will calculate these energies using Density Functional Theory (DFT) with different exchange-correlation functionals. Graphite is characterised by graphene layers with strongly bound carbon atoms in hexagonal rings and weak long-range van der Waals interactions between the layers.

In the sections below we will calculate the lowest energy structures of each compound. This is needed to determine the intercalation energy.

- [Graphite](#graphite)
- [Li metal](#limetal)
- [Li intercalation](#liintercalation)



<a id='graphite'></a>
## Graphite

![graphite](C64.png)

To try some of the methods that we are going to use we will start out by utilizing the interatomic potential [EMT](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html) (Effective Medium Theory) calculator. This will allow to quickly test some of the optimization procedures in ASE, and ensure that the scripts do what they are supposed to, before we move on to the more precise and time consuming DFT calculations. Initially we will calculate the C-C distance and interlayer spacing in graphite in a two step procedure.

In [1]:
# The graphite structure is set up using a tool in ASE
from ase.lattice.hexagonal import Graphite

# One has only to provide the lattice constants
structure = Graphite('C', latticeconstant={'a': 1.5, 'c': 4.0})

To veriry that the structures is as expected we can check it visually.

In [3]:
from ase.visualize import view

view(structure)  # This will pop up the ase gui window

Next we will use the EMT calculator to get the energy of graphite. Remember absolute energies are not meaningful, we will only use energy differences.

In [ ]:
from ase.calculators.emt import EMT

calc = EMT()
structure.set_calculator(calc)
energy = structure.get_potential_energy()
print('Energy of graphite: {0:.2f} eV'.format(energy))


The cell below requires some input. We set up a loop that should calculate the energy of graphite for a series of C-C distances.

In [13]:
import numpy as np

# Provide some initial guesses on the distances
ccdist = 1.3
layerdist = 3.7

# We will explore distances around the initial guess
dists = np.linspace(ccdist * .8, ccdist * 1.2, 10)
# The resulting energies are put in the following list
energies = []
for cc in dists:
    # Calculate the lattice parameters a and c from the C-C distance
    # and interlayer distance, respectively
    a = cc * np.sqrt(3)
    c = 2 * layerdist

    gra = Graphite('C', latticeconstant={'a': a, 'c': c})

    # Set the calculator and attach it to the Atoms object with
    # the graphite structure
    calc = EMT()
    gra.set_calculator(calc)

    energy = gra.get_potential_energy()
    # Append results to the list
    energies.append(energy)
    

Determine the equilibrium lattice constant. We use the [equation of state module](https://wiki.fysik.dtu.dk/ase/ase/eos.html) in ASE.

In [14]:
from ase.eos import EquationOfState

eos = EquationOfState(dists, energies)
v0, e0, B = eos.fit()
ccdist = v0
print(ccdist)


1.322942721169989


![Grapite fit](gra_fit.png)

Make a script that calculates the interlayer distance with EMT in the cell below

In [18]:
# This script will calculate the energy of graphite for a series of inter-layer distances.
from ase.calculators.emt import EMT
from ase.lattice.hexagonal import Graphite
from ase.eos import EquationOfState
import numpy as np

# ccdist is already defined in the previous cell
# Start from a small guess
layerdist = 2.0

# teacher
dists = np.linspace(layerdist * .8, layerdist * 1.2, 10)
# The resulting energies are put in the following list
energies = []
for ld in dists:
    # Calculate the lattice parameters a and c from the C-C distance
    # and interlayer distance, respectively
    a = ccdist * np.sqrt(3)
    c = 2 * ld

    gra = Graphite('C', latticeconstant={'a': a, 'c': c})

    # Set the calculator and attach it to the Atoms object with
    # the graphite structure
    calc = EMT()
    gra.set_calculator(calc)

    energy = gra.get_potential_energy()
    # Append results to the list
    energies.append(energy)
eos = EquationOfState(dists, energies)
v0, e0, B = eos.fit()
layerdist = v0
print(layerdist)


1.8276194864509687


Now both the optimal C-C distance and the interlayer distance is evaluated with EMT. Try to compare with the experimental numbers provided below.

|                         | Experimental values | EMT | LDA | PBE | BEEF-vdW |
|-------------------------|---------------------|-----|-----|-----|----------|
| C-C  distance / Å       |               1.420 |     |     |     |          |
| Interlayer distance / Å |                3.35 |     |     |     |          |


Not surprisingly we need to use more sophisticated theory to model this issue.

Below we will use DFT as implemented in GPAW to determine the same parameters.

First we set up an initial guess of the structure as before.

In [5]:
import numpy as np
from ase.lattice.hexagonal import Graphite

ccdist = 1.40
layerdist = 3.33
a = ccdist * np.sqrt(3)
c = 2 * layerdist
gra = Graphite('C', latticeconstant={'a': a, 'c': c})


Then we create the GPAW calculator object. The parameters are explained [here](https://wiki.fysik.dtu.dk/gpaw/documentation/manual.html#parameters), see especially the section regarding [mode](https://wiki.fysik.dtu.dk/gpaw/documentation/manual.html#manual-mode). This graphite structure has a small unit cell, thus plane wave mode, `mode=PW()`, will be faster than the grid mode, `mode='fd'`. Plane wave mode lets us calculate the strain on the unit cell - useful for optimizing the lattice parameters.

We will start by using the LDA exchange-correlation functional. Later you will try other functionals.

In [6]:
from gpaw import GPAW, PW

xc = 'LDA'
calc = GPAW(mode=PW(500), kpts=(10, 10, 6), xc=xc,
            txt='graphite-{0}.log'.format(xc))

gra.set_calculator(calc)  # Connect system and calculator
print(gra.get_potential_energy())


-40.450597563379446


Then we optimize the unit cell of the structure. We will take advantage of the [StrainFilter](https://wiki.fysik.dtu.dk/ase/ase/constraints.html#the-strainfilter-class) class. This allows us to simultaneously optimize both C-C distance and interlayer distance. We employ the [BFGS](http://aria42.com/blog/2014/12/understanding-lbfgs) algorithm to minimize the strain on the unit cell.

In [7]:
from ase.constraints import StrainFilter
from ase.optimize.bfgs import BFGS
from ase.io import Trajectory

sf = StrainFilter(gra, mask=[1, 1, 1, 0, 0, 0])
opt = BFGS(sf)
traj = Trajectory('graphite-{0}.traj'.format(xc), 'w', gra)
opt.attach(traj)
opt.run(fmax=0.01)


      Step     Time          Energy         fmax
BFGS:    0 15:14:05      -40.450598        3.1769


BFGS:    1 15:15:21      -40.375703        7.6672


BFGS:    2 15:16:45      -40.473027        0.3147


BFGS:    3 15:18:18      -40.472716        0.2172


BFGS:    4 15:19:28      -40.470119        0.3920


BFGS:    5 15:20:37      -40.468641        0.2017


BFGS:    6 15:21:47      -40.468799        0.0285


BFGS:    7 15:22:56      -40.468898        0.0008


True

Read in the result of the relaxation and determine the C-C and interlayer distances.

In [12]:
from ase.io import read
import numpy as np

atoms = read('graphite-{0}.traj'.format(xc))
a = atoms.cell[0][0]
h = atoms.cell[2][2]
# Determine the rest from here
print(a / np.sqrt(3))
print(h / 2)

1.4107987528380115
3.2140468702671305


Now we need to try a GGA type functional (e.g. PBE) and a functional taking into account van der Waals forces (e.g. BEEF-vdw). These functionals will require more computational time, thus the following might be beneficial to read.

If the relaxation takes too long time, we can submit it to be run in parallel on the computer cluster. There are two ways to achieve this:
    
A. Log in to the gbar terminal, save a full script in a file (e.g. `calc.py`) and submit that file to be calculated in parallel (e.g. `mpirun gpaw-python calc.py`).

or

B. Stay in the notebook and make use of the functions supplied in `submit_functions`, these are explained in the following cells.

Make a cell with the full script, it needs to be able to run by itself, i.e. not depend on variables or imports in other cells. A good way to test this is to restart the kernel (this will wipe all variables) and run the cell. Once it has been established that the DFT calculation has been initiated without error, interrupt the kernel and take note of the number in brackets, i.e. x in `In [ x ]`, this is needed to get the text in the cell for submission.

In [ ]:
# Full script
# from ase...

In [1]:
# teacher
import numpy as np
from ase.lattice.hexagonal import Graphite

ccdist = 1.40
layerdist = 3.33
a = ccdist * np.sqrt(3)
c = 2 * layerdist
gra = Graphite('C', latticeconstant={'a': a, 'c': c})

from gpaw import GPAW, PW

xc = 'LDA'
calc = GPAW(mode=PW(500), kpts=(10, 10, 6), xc=xc,
            txt='graphite-{0}.log'.format(xc))

gra.set_calculator(calc)  # Connect system and calculator
from ase.constraints import StrainFilter
from ase.optimize.bfgs import BFGS
from ase.io import Trajectory

sf = StrainFilter(gra, mask=[1, 1, 1, 0, 0, 0])
opt = BFGS(sf)
traj = Trajectory('graphite-{0}.traj'.format(xc), 'w', gra)
opt.attach(traj)
opt.run(fmax=0.01)


KeyboardInterrupt: 

The following cell submits a calculation. The submit function takes two parameters; a name for the calculation and a magic variable starting with "_i" and then the number of the cell that contains the standalone script. The function will create a folder with the supplied name. All output from the calculation will be placed in that folder.

In [2]:
%load_ext autoreload

%autoreload 2

from submit_functions import submit
submit('graphite', _i1)  # 1 is the number of the cell with the calculation script

The calculation "graphite" was submitted with the job ID: 6262766


Check the status of the calculation by supplying the job ID (get the job ID from the output of the submit command). The column with `S` heading gives you a character code of the status: `Q`: still in queue, `R`: running, `C`: finished.

In [ ]:
from submit_functions import check_status
check_status(123456)  # 123456 is the job ID from the output of the submit command


When the calculation finishes the result can be interpreted by [reading](https://wiki.fysik.dtu.dk/ase/ase/io/io.html#ase.io.read) in the Atoms object. This assumes that the name `graphite` was supplied when submitting and the trajectory file is called: `graphite-LDA.traj`, if not change accordingly.

In [ ]:
from ase.io import read

# The full relaxation
relaxation = read('graphite/graphite-LDA.traj', index=':')
# The final image of the relaxation containing the relevant energy
atoms = read('graphite/graphite-LDA.traj')

In [ ]:
# Extract the relevant information from the calculation
# Energy
print(atoms.get_potential_energy())
# Unit cell
print(atoms.get_cell())
# See the steps of the optimization
from ase.visualize import view
view(relaxation)

<a id='limetal'></a>
## Li metal

![Li metal](Li2.png)

Now we need to calculate the energy of Li metal. We will use the same strategy as for graphite, this time though you will have to do most of the work.

Some hints:
1. The crystal structure of Li metal is shown in the image above. That structure is easily created with one of the functions in [ase.lattice](https://wiki.fysik.dtu.dk/ase/ase/lattice.html#available-crystal-lattices)
2. A k-point density of approximately 2.4 points / Angstrom will be sufficient
3. Use a FermiDirac occupation (TEST THIS or just give a GPAW object)

In the end try to compare the different functionals with experimental values:

|             | Experimental values | LDA | PBE | BEEF-vdW |
|-------------|---------------------|-----|-----|----------|
| a / Å       |                3.51 |     |     |          |



In [14]:
# teacher

from ase import Atoms
from gpaw import GPAW, FermiDirac, PW
from ase.optimize import QuasiNewton
from ase.lattice.cubic import BodyCenteredCubic
from ase.constraints import StrainFilter

# This script will optimize lattice constant of metallic lithium

xc = 'LDA'  # xc-functional. Change to 'PBE' or 'BEEF-vdw' to run with other functionals
Li_metal = BodyCenteredCubic(symbol='Li', latticeconstant=3.3)

calc = GPAW(mode=PW(500),
            kpts=(8, 8, 8),
            occupations=FermiDirac(0.15),
            nbands=-10,
            txt='Li-metal-{0}.log'.format(xc),
            xc=xc)

Li_metal.set_calculator(calc)

sf = StrainFilter(Li_metal, mask=[1, 1, 1, 0, 0, 0])
opt = BFGS(sf)
opt.run(fmax=0.01)



      Step     Time          Energy         fmax
BFGS:    0 13:38:58       -3.764298        0.8537
BFGS:    1 13:39:15       -3.773624        0.7020
BFGS:    2 13:39:32       -3.791540        0.2397
BFGS:    3 13:39:47       -3.793876        0.0200
BFGS:    4 13:40:03       -3.793872        0.0006


True

<a id='liintercalation'></a>
## Li intercalation in graphite

![Li intercalated in graphite](C144Li18.png)

Now we will calculate the intercalation of Li in graphite. For simplicity we will represent the graphite with only one layer. Also try and compare the C-C and interlayer distances to experimental values.

|                         | Experimental values | LDA | PBE | BEEF-vdW |
|-------------------------|---------------------|-----|-----|----------|
| C-C  distance / Å       |               1.441 |     |     |          |
| Interlayer distance / Å |               3.706 |     |     |          |


In [ ]:
from ase.lattice.hexagonal import Graphene

ccdist = 1.40
layerdist = 3.7

a = ccdist * np.sqrt(3)
c = layerdist

# We will require a larger cell, to accomodate the Li
Li_gra = Graphene('C', size=(2, 2, 1), latticeconstant={'a': a, 'c': c})
# Append an Li atom on top of the graphene layer
Li_gra.append(Atom('Li', (a / 2, ccdist / 2, layerdist / 2)))


In [ ]:
# teacher

from ase import Atoms
from gpaw import GPAW, PW
from ase import Atom
from ase.optimize.bfgs import BFGS
import numpy as np
from ase.visualize import view
from ase.lattice.hexagonal import Graphene

ccdist = 1.40
layerdist = 3.7

a = ccdist * np.sqrt(3)
c = layerdist

# We will require a larger cell, to accomodate the Li
Li_gra = Graphene('C', size=(2, 2, 1), latticeconstant={'a': a, 'c': c})
Li_gra.append(Atom('Li', (a / 2, ccdist / 2, layerdist / 2)))

calc = GPAW(mode=PW(500), kpts=(5, 5, 6), xc=xc, txt=None)

Li_gra.set_calculator(calc)  # Connect system and calculator

sf = StrainFilter(gra, mask=[1, 1, 1, 0, 0, 0])
opt = BFGS(sf)
opt.run(fmax=0.01)

print(Li_gra.cell)

In [ ]:
# teacher
from ase import Atoms
from gpaw import GPAW, PW
from ase import Atom
from ase.optimize.bfgs import BFGS
import numpy as np
from ase.visualize import view
from ase.lattice.hexagonal import Graphene
from ase.constraints import StrainFilter

ccdist = 1.40
layerdist = 3.7

a = ccdist * np.sqrt(3)
c = layerdist
Li_gra = Atoms('CCCCCCLi', positions=[(0, 0, 0), (0, ccdist, 0), (a, 0, 0),
                                   (-a, 0, 0), (-a / 2, -ccdist / 2, 0),
                                   (a / 2, -ccdist / 2, 0), (0, -ccdist, c / 2)],
            cell=([1.5 * a, -1.5 * ccdist, 0],
                  [1.5 * a, 1.5 * ccdist, 0],
                  [0, 0, c]),
            pbc=(1, 1, 1))
calc = GPAW(mode=PW(500), kpts=(5, 5, 6), xc=xc, txt=None)

Li_gra.set_calculator(calc)  # Connect system and calculator

sf = StrainFilter(Li_gra, mask=[1, 1, 1, 0, 0, 0])
opt = BFGS(sf)
opt.run(fmax=0.01)


Now calculate the intercalation energy of Li in graphite with the following formula:
$$E_{Li@graphite} = E_{LiC_6} - (E_{Li(s)} + E_{C_6^{graphite}})$$
Remember to adjust the energies so that the correct number of atoms is taken into account.

These are the experimental values to compare with
In the end try to compare the different functionals with experimental values:

|                           | Experimental values | LDA | PBE | BEEF-vdW |
|---------------------------|---------------------|-----|-----|----------|
| Intercalation energy / eV |              -0.124 |     |     |          |


In [ ]:
# teacher
e_Li_gra = Li_gra.get_potential_energy()
e_Li = Li_metal.get_potential_energy() / len(Li_metal)
e_C6 = 6 * gra.get_potential_energy() / len(gra)
intercalation_energy = e_Li_gra - (e_Li + e_C6)
print('Intercalation energy: {0:.2f}eV'.format(intercalation_energy))

Great job! When you have made it this far it is time to turn your attention to the cathode.